Fatima Nasyr

Affordable Housing Production by Building

In [ ]:
!pip install plotly

import pandas as pd
import numpy as np
import json
import plotly.express as px
import plotly.io as pio

pio.renderers.default = "colab"


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Affordable_Housing_Production_by_Building.csv to Affordable_Housing_Production_by_Building.csv


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving airbnb_clean.csv to airbnb_clean (1).csv


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving nycc.json to nycc (1).json


In [ ]:
aff = pd.read_csv("Affordable_Housing_Production_by_Building.csv")
air = pd.read_csv("airbnb_clean.csv")

with open("nycc.json", "r") as f:
    nycc = json.load(f)


In [ ]:
aff["Council District"] = pd.to_numeric(aff["Council District"], errors="coerce")

aff_district = (
    aff.dropna(subset=["Council District"])
       .assign(District_str=lambda df: df["Council District"].astype(int).astype(str).str.zfill(2))
       .groupby("District_str", as_index=False)
       .agg(
            affordable_units=("All Counted Units", "sum"),
            borough=("Borough", lambda s: s.mode().iloc[0] if len(s.mode()) else "Unknown"),
            projects=("Project ID", "nunique")
       )
)

aff_district.head()


,District_str,affordable_units,borough,projects
0,01,5981.0,Manhattan,39
1,02,5956.0,Manhattan,59
2,03,8010.0,Manhattan,64
3,04,6741.0,Manhattan,24
4,05,1543.0,Manhattan,19


In [ ]:
def point_in_ring(lon, lat, ring):
    inside = False
    n = len(ring)
    for i in range(n - 1):
        x1, y1 = ring[i]
        x2, y2 = ring[i+1]
        if ((y1 > lat) != (y2 > lat)):
            x_intersect = x1 + (lat - y1) * (x2 - x1) / ((y2 - y1) + 1e-12)
            if lon < x_intersect:
                inside = not inside
    return inside

def point_in_polygon(lon, lat, polygon):
    if not point_in_ring(lon, lat, polygon[0]):
        return False
    for hole in polygon[1:]:
        if point_in_ring(lon, lat, hole):
            return False
    return True

def lookup_district_for_point(lon, lat, features):
    for feat in features:
        dist = feat["properties"]["District"]
        geom = feat["geometry"]

        if geom["type"] == "Polygon":
            if point_in_polygon(lon, lat, geom["coordinates"]):
                return dist

        elif geom["type"] == "MultiPolygon":
            for poly in geom["coordinates"]:
                if point_in_polygon(lon, lat, poly):
                    return dist
    return None


In [ ]:
air_valid = air.dropna(subset=["latitude", "longitude"]).copy()
features = nycc["features"]

def get_dist(row):
    return lookup_district_for_point(row["longitude"], row["latitude"], features)

# This may take a few seconds depending on dataset size
air_valid["District_str"] = air_valid.apply(get_dist, axis=1)

air_valid["District_str"].value_counts().head()


,count
District_str,
04,2787
03,2694
36,2364
34,2323
01,2223


In [ ]:
if "id" in air_valid.columns:
    id_col = "id"
else:
    air_valid = air_valid.reset_index().rename(columns={"index": "listing_id"})
    id_col = "listing_id"

air_district = (
    air_valid.dropna(subset=["District_str"])
             .groupby("District_str", as_index=False)
             .agg(airbnb_listings=(id_col, "nunique"))
)

air_district.head()


,District_str,airbnb_listings
0,01,2223
1,02,2213
2,03,2694
3,04,2787
4,05,1184


In [ ]:
combined = pd.merge(
    aff_district,
    air_district,
    on="District_str",
    how="outer"
)

for col in ["affordable_units", "projects", "airbnb_listings"]:
    combined[col] = combined[col].fillna(0)

combined["ratio"] = np.where(
    combined["affordable_units"] > 0,
    combined["airbnb_listings"] / combined["affordable_units"],
    np.nan
)

def classify(row, median_aff):
    if row["affordable_units"] == 0 and row["airbnb_listings"] == 0:
        return "Low activity"
    if np.isnan(row["ratio"]):
        if row["airbnb_listings"] > 0:
            return "Tourism (0 affordable units)"
        return "Low activity"
    if row["ratio"] > 1.5:
        return "Tourism-leaning"
    if row["ratio"] < 0.5 and row["affordable_units"] > median_aff:
        return "Resident-leaning"
    return "Mixed"

median_aff = combined["affordable_units"].replace(0, np.nan).median()

combined["strategy"] = combined.apply(classify, axis=1, median_aff=median_aff)

combined.head()


,District_str,affordable_units,borough,projects,airbnb_listings,ratio,strategy
0,01,5981.0,Manhattan,39,2223,0.371677,Resident-leaning
1,02,5956.0,Manhattan,59,2213,0.371558,Resident-leaning
2,03,8010.0,Manhattan,64,2694,0.336330,Resident-leaning
3,04,6741.0,Manhattan,24,2787,0.413440,Resident-leaning
4,05,1543.0,Manhattan,19,1184,0.767336,Mixed


In [ ]:
fig = px.scatter(
    combined,
    x="affordable_units",
    y="airbnb_listings",
    color="strategy",
    hover_name="District_str",
    log_x=True,
    log_y=True,
    title="NYC Council Districts: Affordable Housing vs Airbnb Activity",
    labels={
        "affordable_units": "Affordable Units (log scale)",
        "airbnb_listings": "Airbnb Listings (log scale)"
    }
)

fig.show()


In [ ]:
borough_colors = {
    "Queens": "#1f77b4",       # Blue
    "Brooklyn": "#ff7f0e",     # Orange
    "Manhattan": "#2ca02c",    # Green
    "Bronx": "#d62728",        # Red
    "Staten Island": "#9467bd",# Purple
    "Unknown": "gray"
}


In [ ]:
fig_bar_tourism = px.bar(
    tourism_top,
    x="District_str",
    y="ratio",
    color="borough",
    text="ratio",
    title="Top 10 Tourism-Leaning Districts<br><sup>Highest Airbnb listings per affordable unit</sup>",
    labels={
        "District_str": "Council District",
        "ratio": "Airbnb Listings per Affordable Unit"
    },
    color_discrete_map=borough_colors
)

fig_bar_tourism.update_traces(texttemplate="%{text:.2f}", textposition="outside")


NameError: name 'tourism_top' is not defined

In [ ]:
ratio_valid = combined.dropna(subset=["ratio"])

fig = px.box(
    ratio_valid,
    x="borough",
    y="ratio",
    title="Distribution of Airbnb-to-Affordable Ratio by Borough",
    points="all",
    log_y=True,
    labels={"ratio": "Airbnb Listings per Affordable Unit"}
)

fig.show()


In [ ]:
counts = combined["strategy"].value_counts().reset_index()
counts.columns = ["strategy", "count"]

fig = px.bar(
    counts,
    x="strategy",
    y="count",
    text="count",
    title="How Districts Are Classified (Resident vs Tourism vs Mixed)"
)

fig.update_traces(textposition="outside")
fig.show()


In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig_combined = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        fig_bar_tourism.layout.title.text,
        fig_bar_resident.layout.title.text
    )
)

# Add tourism chart traces
for trace in fig_bar_tourism.data:
    fig_combined.add_trace(trace, row=1, col=1)

# Add resident chart traces
for trace in fig_bar_resident.data:
    fig_combined.add_trace(trace, row=1, col=2)

# Final layout
fig_combined.update_layout(
    height=600,
    width=1400,
    showlegend=True,
    title_text="Comparison: Tourism-Leaning vs Resident-Leaning Districts",
    title_x=0.5
)

# Axis labels
fig_combined.update_xaxes(title_text="Council District", row=1, col=1)
fig_combined.update_yaxes(title_text="Airbnb per Affordable Unit", row=1, col=1)

fig_combined.update_xaxes(title_text="Council District", row=1, col=2)
fig_combined.update_yaxes(title_text="Affordable Housing Units", row=1, col=2)

fig_combined.show()


NameError: name 'fig_bar_tourism' is not defined

In [ ]:
# ============================================================
# PATHS (change if needed)
# ============================================================
# Original Windows path for reference:
# C:\Users\Windows\Desktop\visual-analytics-final-project-main\Affordable_Housing_Production_by_Building.csv

AFF_PATH   = "/content/Affordable_Housing_Production_by_Building.csv"
AIR_PATH   = "/content/airbnb_clean.csv"
NYCC_PATH  = "/content/nycc.json"

# ============================================================
# LOAD DATA
# ============================================================
aff = pd.read_csv(AFF_PATH)
air = pd.read_csv(AIR_PATH)

with open(NYCC_PATH, "r") as f:
    nycc_geojson = json.load(f)

print("Affordable housing shape:", aff.shape)
print("Airbnb shape:", air.shape)
print("NYCC feature count:", len(nycc_geojson["features"]))


Affordable housing shape: (8604, 41)
Airbnb shape: (36111, 20)
NYCC feature count: 51


In [ ]:
# ============================================================
# HELPER: POINT IN POLYGON (RAY CASTING)
# ============================================================

def point_in_ring(lon, lat, ring):
    """
    Return True if (lon, lat) is inside a polygon ring (list of [lon, lat]).
    Ray casting algorithm.
    """
    inside = False
    n = len(ring)
    for i in range(n - 1):
        x1, y1 = ring[i]
        x2, y2 = ring[i + 1]
        # Check if edge intersects a horizontal line at 'lat'
        if ((y1 > lat) != (y2 > lat)):
            # Compute x coordinate where edge crosses that horizontal line
            x_intersect = x1 + (lat - y1) * (x2 - x1) / (y2 - y1 + 1e-12)
            if lon < x_intersect:
                inside = not inside
    return inside

def point_in_polygon(lon, lat, polygon_coords):
    """
    polygon_coords: [ [ring1], [ring2], ... ]
    ring1 is outer ring, others (if any) are holes.
    """
    if not polygon_coords:
        return False
    # Must be in outer ring
    if not point_in_ring(lon, lat, polygon_coords[0]):
        return False
    # Must NOT be in any hole
    for hole in polygon_coords[1:]:
        if point_in_ring(lon, lat, hole):
            return False
    return True

def get_district_for_point(lon, lat, features):
    """
    Loop over all council district features and return the 'District'
    for the first polygon that contains the point.
    """
    for feat in features:
        props = feat["properties"]
        geom  = feat["geometry"]
        dist_code = props.get("District")  # e.g. "01", "02", ..

        if geom["type"] == "Polygon":
            if point_in_polygon(lon, lat, geom["coordinates"]):
                return dist_code

        elif geom["type"] == "MultiPolygon":
            for poly_coords in geom["coordinates"]:  # each poly has its own rings
                if point_in_polygon(lon, lat, poly_coords):
                    return dist_code
    return None


In [ ]:
# Keep only rows with valid lat/lon
air_valid = air.dropna(subset=["latitude", "longitude"]).copy()

# OPTIONAL: you can filter to last 11 months ahead of time if your file has a date column.
# For now, we assume airbnb_clean already filtered to last 11 months as you said.

# Apply the district lookup
features = nycc_geojson["features"]

def lookup_district(row):
    return get_district_for_point(
        lon=row["longitude"],
        lat=row["latitude"],
        features=features
    )

# This step can take some time on large datasets
air_valid["District_str"] = air_valid.apply(lookup_district, axis=1)

air_valid["District_str"].value_counts().head()


,count
District_str,
04,2787
03,2694
36,2364
34,2323
01,2223


In [ ]:
air_district = (
    air_valid
    .dropna(subset=["District_str"])
    .groupby("District_str", as_index=False)
    .agg(
        airbnb_listings=("id", "nunique"),
        # approximate "Airbnb intensity" by total estimated revenue
        airbnb_total_revenue=("estimated_revenue_l365d", "sum")
    )
)

air_district.head()


,District_str,airbnb_listings,airbnb_total_revenue
0,01,2223,62348336.0
1,02,2213,24618425.0
2,03,2694,46138628.0
3,04,2787,54008257.0
4,05,1184,10486769.0


In [ ]:
# Merge on District_str
combined = pd.merge(
    aff_district,
    air_district,
    on="District_str",
    how="outer",
    validate="1:1"
)

# Replace NaNs with 0 for counts
for col in ["affordable_units", "projects", "airbnb_listings", "airbnb_total_revenue"]:
    combined[col] = combined[col].fillna(0)

# Avoid division by zero
combined["ratio_airbnb_per_affordable"] = np.where(
    combined["affordable_units"] > 0,
    combined["airbnb_listings"] / combined["affordable_units"],
    np.nan
)

# Simple rule-based strategy label
def classify_row(row):
    a = row["affordable_units"]
    b = row["airbnb_listings"]
    r = row["ratio_airbnb_per_affordable"]

    if a == 0 and b == 0:
        return "Low activity"
    if pd.isna(r):
        # No affordable units but some Airbnb
        if b > 0:
            return "Tourism-leaning (no affordable units)"
        else:
            return "Low activity"
    # Use ratio thresholds – you can tweak
    if r > 1.5:
        return "Tourism-leaning"
    elif r < 0.5 and a > np.median(combined["affordable_units"]):
        return "Resident-leaning"
    else:
        return "Mixed"

combined["strategy"] = combined.apply(classify_row, axis=1)

combined.sort_values("ratio_airbnb_per_affordable", ascending=False).head()


,District_str,affordable_units,borough,projects,airbnb_listings,airbnb_total_revenue,ratio_airbnb_per_affordable,strategy
24,25,196.0,Queens,34,487,1842853.0,2.484694,Tourism-leaning
18,19,86.0,Queens,28,141,1428718.0,1.639535,Tourism-leaning
29,30,374.0,Queens,25,526,2972320.0,1.406417,Mixed
22,23,71.0,Queens,31,80,672165.0,1.126761,Mixed
4,05,1543.0,Manhattan,19,1184,10486769.0,0.767336,Mixed


In [ ]:
fig_scatter = px.scatter(
    combined,
    x="affordable_units",
    y="airbnb_listings",
    color="strategy",
    size="airbnb_total_revenue",
    hover_name="District_str",
    hover_data=["borough", "projects"],
    title="Council District Strategy: Long-Term Residents vs Short-Term Visitors",
    labels={
        "affordable_units": "Affordable Housing Units (All Counted Units)",
        "airbnb_listings": "Number of Airbnb Listings"
    }
)

fig_scatter.update_layout(
    xaxis=dict(type="log", title="Affordable Units (log scale)"),
    yaxis=dict(type="log", title="Airbnb Listings (log scale)")
)

fig_scatter.show()


In [ ]:
borough_colors = {
    "Queens": "#1f77b4",        # Blue
    "Brooklyn": "#ff7f0e",      # Orange
    "Manhattan": "#2ca02c",    # Green
    "Bronx": "#d62728",        # Red
    "Staten Island": "#9467bd",# Purple
    "Unknown": "gray"
}


In [ ]:
# Top 10 tourism-leaning (by ratio, with at least some units)
tourism_top = (
    combined
    .query("affordable_units > 0 and airbnb_listings > 0")
    .sort_values("ratio_airbnb_per_affordable", ascending=False)
    .head(10)
)

fig_bar_tourism = px.bar(
    tourism_top,
    x="District_str",
    y="ratio_airbnb_per_affordable",
    color="borough",
    title="Top 10 Tourism-Leaning Council Districts<br><sub>Highest ratio of Airbnb listings per affordable unit</sub>",
    labels={
        "District_str": "Council District",
        "ratio_airbnb_per_affordable": "Airbnb Listings per Affordable Unit"
    },
    color_discrete_map=borough_colors   # ✅ THIS LINE MAKES COLORS FIXED
)

fig_bar_tourism.show()


In [ ]:
# Top 10 resident-leaning by affordable_units
resident_top = (
    combined
    .sort_values("affordable_units", ascending=False)
    .head(10)
)

fig_bar_resident = px.bar(
    resident_top,
    x="District_str",
    y="affordable_units",
    color="borough",
    title="Top 10 Resident-Leaning Council Districts<br><sub>Highest affordable housing units</sub>",
    labels={
        "District_str": "Council District",
        "affordable_units": "Affordable Housing Units"
    },
    color_discrete_map=borough_colors   # ✅ SAME MAP USED HERE
)

fig_bar_resident.show()


In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Create 1 row, 2 column subplot
fig_combined = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        fig_bar_tourism.layout.title.text,
        fig_bar_resident.layout.title.text
    )
)

# ---------------------------
# Add Tourism-Leaning Traces
# ---------------------------
for trace in fig_bar_tourism.data:
    fig_combined.add_trace(trace, row=1, col=1)

# ---------------------------
# Add Resident-Leaning Traces
# ---------------------------
for trace in fig_bar_resident.data:
    fig_combined.add_trace(trace, row=1, col=2)

# ---------------------------
# Update Layout
# ---------------------------
fig_combined.update_layout(
    height=600,
    width=1400,
    showlegend=True,
    title_text="Comparison of Tourism-Leaning vs Resident-Leaning Council Districts",
    title_x=0.5
)

# Apply axis titles
fig_combined.update_xaxes(title_text="Council District", row=1, col=1)
fig_combined.update_yaxes(title_text="Airbnb per Affordable Unit", row=1, col=1)

fig_combined.update_xaxes(title_text="Council District", row=1, col=2)
fig_combined.update_yaxes(title_text="Affordable Housing Units", row=1, col=2)

fig_combined.show()
